# 빅데이터 분석 기말고사

# 김준호 201710860

## 문제: 서울시 지하철 호선별 역별 유/무임 승하차 인원 분석



'서울시 지하철 호선별 역별 유/무임 승하차 인원 정보'를 csv파일로 내려받아서 분석하세요

https://data.seoul.go.kr/dataList/OA-12251/S/1/datasetView.do?tab=S

* (1-1) '서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv'를 데이터프레임으로 읽고, 스키마를 출력

In [3]:
import pyspark

spark = pyspark.sql.SparkSession\
    .builder\
    .master('local')\
    .appName('prac')\
    .config(conf=pyspark.SparkConf())\
    .getOrCreate()

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

_schema = StructType([
    StructField(u"사용월", StringType(), True),
    StructField(u"호선명", StringType(), True),
    StructField(u"지하철역", StringType(), True),
    StructField(u"유임승차인원", IntegerType(), True),
    StructField(u"무임승차인원", IntegerType(), True),
    StructField(u"유임하차인원", IntegerType(), True),
    StructField(u"무임하차인원", IntegerType(), True),
    StructField(u"작업일자", StringType(), True),
])

df1 = spark\
    .read\
    .option("charset", "euc-kr")\
    .option("header", "true")\
    .schema(_schema)\
    .csv('final.csv')
df1.show()

+------+------+----------------------+------------+------------+------------+------------+--------+
|사용월|호선명|              지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|
+------+------+----------------------+------------+------------+------------+------------+--------+
|202111| 1호선|                  시청|      516353|       72373|      528021|       72182|20211203|
|202111| 1호선|                동묘앞|      119396|      137406|      128531|      138097|20211203|
|202111| 1호선|                신설동|      271006|      108130|      262798|      103695|20211203|
|202111| 1호선|청량리(서울시립대입구)|      315789|      232661|      317666|      237208|20211203|
|202111| 1호선|                  종각|      797960|      118815|      781329|      110536|20211203|
|202111| 1호선|                제기동|      222630|      247332|      214314|      263992|20211203|
|202111| 1호선|                동대문|      210503|      112597|      195301|      110925|20211203|
|202111| 1호선|               종로3가|      433793|      272242|      397349|      

In [5]:
df1.printSchema()

root
 |-- 사용월: string (nullable = true)
 |-- 호선명: string (nullable = true)
 |-- 지하철역: string (nullable = true)
 |-- 유임승차인원: integer (nullable = true)
 |-- 무임승차인원: integer (nullable = true)
 |-- 유임하차인원: integer (nullable = true)
 |-- 무임하차인원: integer (nullable = true)
 |-- 작업일자: string (nullable = true)



* (1-2) 건수를 출력

In [6]:
print(f"건수: {df1.count()}")

건수: 55171


* (1-3) '호선명'별 '유임승차인원'의 순위를 매기고, 그 결과를 30줄 출력 (마지막 'rank' 컬럼)

In [7]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

df1.withColumn("rank", F.rank().over(Window().partitionBy([u'호선명']).orderBy(F.col(u"유임승차인원").desc()))).show(30)

+------+------+--------+------------+------------+------------+------------+--------+----+
|사용월|호선명|지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+------+--------+------------+------------+------------+------------+--------+----+
|201605|일산선|    화정|      515925|      139163|      535460|      142410|20160608|   1|
|201512|일산선|    화정|      508044|      136734|      529106|      139931|20160108|   2|
|201603|일산선|    화정|      502144|      140596|      526030|      144088|20160408|   3|
|201703|일산선|    화정|      499911|      141327|      529944|      145502|20170403|   4|
|201510|일산선|    화정|      499388|      520595|      140545|      144148|20151108|   5|
|201905|일산선|    화정|      499218|      149790|      520966|      152915|20190603|   6|
|201612|일산선|    화정|      497514|      132807|      521734|      136086|20170108|   7|
|201805|일산선|    화정|      494994|      139239|      515748|      142997|20180603|   8|
|201610|일산선|    화정|      492146|      137462|      513849|      140303|201611

* (1-4) '호선명'별 '유임승차인원'이 순위가 가장 높은 경우만 30건 출력 (마지막 'rank' 컬럼이 모두 1)

In [8]:
df1.withColumn(
    "rankByLine", 
    F.rank().over(Window().partitionBy([u'호선명']).orderBy(F.col(u"유임승차인원").desc())))\
    .filter(F.col("rankByLine")==1)\
    .show(30)

+------+--------------+--------------------+------------+------------+------------+------------+--------+----------+
|사용월|        호선명|            지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rankByLine|
+------+--------------+--------------------+------------+------------+------------+------------+--------+----------+
|201605|        일산선|                화정|      515925|      139163|      535460|      142410|20160608|         1|
|201905|        장항선|                아산|      109243|       12901|       93737|       11808|20190603|         1|
|201512|        경부선|              영등포|     1439049|      280623|     1545631|      280969|20160108|         1|
|201905|    우이신설선|        북한산보국문|      161867|       45041|      141742|       45814|20190603|         1|
|201512|        분당선|                야탑|      769442|      157846|      809197|      156034|20160108|         1|
|201512|         7호선|      가산디지털단지|     1245731|       82588|     1243464|       78424|20160108|         1|
|201905|        수인선|      

* (1-5) '호선명'별 '유임승차인원'의 zscore를 계산하고 출력. 윈도우 함수를 사용하여야 한다.

In [9]:
import sys

df1 = df1.withColumn("meanByLine", F.avg(df1['유임승차인원']).over(Window.rowsBetween(-sys.maxsize, sys.maxsize)))
df1 = df1.withColumn("sdByLine", F.stddev(df1['유임승차인원']).over(Window.rowsBetween(-sys.maxsize, sys.maxsize)))
df1 = df1.withColumn("zscore", (F.col('유임승차인원')-F.col('meanByLine'))/F.col('sdByLine'))
df1.select(['호선명', '유임승차인원', 'meanByLine', 'sdByLine', 'zscore']).show()

21/12/18 22:35:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/18 22:35:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+------------+-----------------+-----------------+--------------------+
|호선명|유임승차인원|       meanByLine|         sdByLine|              zscore|
+------+------------+-----------------+-----------------+--------------------+
| 1호선|      516353|302546.6513023146|322895.6016002317|  0.6621531778013912|
| 1호선|      119396|302546.6513023146|322895.6016002317| -0.5672132119317885|
| 1호선|      271006|302546.6513023146|322895.6016002317|-0.09768064707602991|
| 1호선|      315789|302546.6513023146|322895.6016002317| 0.04101123902604408|
| 1호선|      797960|302546.6513023146|322895.6016002317|  1.5342833604498682|
| 1호선|      222630|302546.6513023146|322895.6016002317| -0.2474999687399187|
| 1호선|      210503|302546.6513023146|322895.6016002317|-0.28505699937118173|
| 1호선|      433793|302546.6513023146|322895.6016002317|  0.4064668210011047|
| 1호선|      362377|302546.6513023146|322895.6016002317| 0.18529316720690342|
| 1호선|     1086374|302546.6513023146|322895.6016002317|  2.4274946602342413|
| 

### 문제2: 회귀분석

sklearn의 make_regression함수를 다음과 같이 사용하여 데이터를 생성하고, 문제를 푸세요.



```

from sklearn.datasets import make_regression

X, y, coef = make_regression(n_samples = 200,

                            n_features = 4,

                            n_informative = 3,

                            n_targets = 1,

                            noise = 0.0,

                            coef = True) ... [식-1]

```

* (2-1) gradient 방법으로 회귀식을 계산하고, 계수를 출력.

또한 [식-1]의 'coef'를 출력하고, gradient방법에서 도출된 계수와 일치하는지 비교하세요.

주의: 반복이 많으면 감점 (예를 들어, 1000 이상이면 많다고 간주할 것임)

In [10]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_samples = 200,
                            n_features = 4,
                            n_informative = 3,
                            n_targets = 1,
                            noise = 0.0,
                            coef = True)

In [12]:
import numpy as np
import pandas as pd

y = y.reshape(-1, 1)

Xy = np.concatenate([X, y], axis=1)
Xy.shape

df2Pd = pd.DataFrame(Xy, columns=['0', '1', '2', '3', 'target'])
df2 = spark.createDataFrame(df2Pd)
df2.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+
|                   0|                   1|                   2|                   3|             target|
+--------------------+--------------------+--------------------+--------------------+-------------------+
|  0.6899018738124507|-0.37291104149940385|  0.5729429671982869|  0.9557368637500713|  50.56095973779512|
|  0.7151172003915949| -0.8488178577848516|-0.24890456755799306| -0.3369688376552179| -36.76720738116924|
|  1.5620011151819444|  -0.638702166343831|  -0.560575646986203| -0.6666775554100492| -48.65495210688528|
|-0.25348534629172653| -1.0439139125810506| -0.5144825600500981|  0.7624600466376814| -78.43730598638561|
| -1.2335144463962915|  0.5284489405352963|  1.3412108164535006|  1.4457752693485975| 120.69442959064382|
|  0.8817261230592149|   0.433403650081954|-0.02999503657560...| -0.4144584557681783|  20.23465927616143|
|  0.7568948744569237|-0.15481101887287063| -1

In [13]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

va = VectorAssembler(inputCols=['0', '1', '2', '3'], outputCol='features')
lr = LinearRegression(featuresCol='features', labelCol='target', maxIter=500)

df2_va = va.transform(df2)
model = lr.fit(df2_va)

np.set_printoptions(precision=5)
print(f"coef [식-1]    : {coef}")
print(f"coef gradient, {model.coefficients}")


21/12/18 22:35:43 WARN Instrumentation: [8f0d81ea] regParam is zero, which might cause numerical instability and overfitting.
21/12/18 22:35:43 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/12/18 22:35:43 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/12/18 22:35:43 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/12/18 22:35:43 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


coef [식-1]    : [12.61544 27.30894 90.83164  0.     ]
coef gradient, [12.615440421574885,27.308939936144807,90.83163938689728,-5.83716691046e-15]


### 결과 해석 : 2-1 일치한다.

* (2-2) 스파크 데이터프레임을 생성 (이후 계속 스파크로 푸세요)

In [14]:
import numpy as np

y = y.reshape(-1, 1)
Xy = np.concatenate([X, y], axis=1)

df2Pd = pd.DataFrame(Xy, columns=['0', '1', '2', '3', 'y'])
df2 = spark.createDataFrame(df2Pd)
df2.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+
|                   0|                   1|                   2|                   3|                  y|
+--------------------+--------------------+--------------------+--------------------+-------------------+
|  0.6899018738124507|-0.37291104149940385|  0.5729429671982869|  0.9557368637500713|  50.56095973779512|
|  0.7151172003915949| -0.8488178577848516|-0.24890456755799306| -0.3369688376552179| -36.76720738116924|
|  1.5620011151819444|  -0.638702166343831|  -0.560575646986203| -0.6666775554100492| -48.65495210688528|
|-0.25348534629172653| -1.0439139125810506| -0.5144825600500981|  0.7624600466376814| -78.43730598638561|
| -1.2335144463962915|  0.5284489405352963|  1.3412108164535006|  1.4457752693485975| 120.69442959064382|
|  0.8817261230592149|   0.433403650081954|-0.02999503657560...| -0.4144584557681783|  20.23465927616143|
|  0.7568948744569237|-0.15481101887287063| -1

* (2-3) 훈련, 테스트 데이터를 6:4 비율로 분리하고,

In [15]:
(trainDf2, testDf2) = df2.randomSplit([0.6, 0.4])

* (2-4) 회귀모델링하고 계수와 절편 출력. 

In [16]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

va = VectorAssembler(inputCols=['0', '1', '2', '3'], outputCol='features')
lr = LinearRegression(featuresCol='features', labelCol='y', maxIter=500, regParam=0.3, elasticNetParam=0.8)

df2_va_train = va.transform(trainDf2)
model = lr.fit(df2_va_train)

print(f"coef gradient, {model.coefficients}")

coef gradient, [12.426770528698881,27.053594494296053,90.55245436586932,0.0]


In [17]:
df2_va_lr_train = model.transform(df2_va_train)
df2_va_lr_train.select(['y', 'prediction']).show(10)

+-------------------+-------------------+
|                  y|         prediction|
+-------------------+-------------------+
|-250.36993835361162| -249.2082465044416|
|-120.20063273169158|-119.32763010709401|
|-56.920429652512034| -56.42274120752914|
|  17.72889306757847| 18.182205441893654|
|-159.02983556814215| -158.1225521728087|
|-170.58159572661899|-169.72854272056634|
| -58.02950823121105|-57.771725669035696|
|  74.98983142445692|  75.14476596442303|
|-52.957884011685906|  -52.5330550424589|
| -26.88822797562444|-26.553378662016378|
+-------------------+-------------------+
only showing top 10 rows



* (2-5) 테스트 데이터에 대해 R2를 구해서 출력하세요.

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator

df2_va_test = va.transform(testDf2)
df2_va_lr_test = model.transform(df2_va_test)

evaluator = RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(df2_va_lr_test)

print(f"r2: {r2}")

r2: 0.999979032697076


### 문제3: 텍스트 분류

In [19]:
%%writefile final_negative.txt
- 고구마 먹은 영화 유치해서 못보겠다

- 노잼이네요.. 왜 배우들이 출연했는지 이해가 잘 안됨

- 제발 뻘짓 좀 그만하셨으면.... 이건 여러모로 낭비다.

- 스토리가 매우 매우 아쉽다

- 영화 약간 이해가 안됨

- 좋아하는 배우들인데 최근 영화라는 생각이 안 드는 올드한 연출과 스토리인듯 해 아쉽다

- 그지같네 도대체 이 따위 영화를 그렇게 비밀로 붙이면서 비공개로 한 이유가 뭐냐? 이걸 영화라고 쳐 만든거냐? 최악의 마블 영화 중에 영화닷

- 스크린보다 시계를 더 많이 봄.. 중구난방 유치 폭발

- 이때까지 본 영화중에 제일 최악 재미도 없고 그냥 소리만 빽빽 지름 돈많은 초등학생이 맘대로 영화찍은 느낌 연기도 못함

- 나만 짜증.. 줄거리 한줄평.. 제발 말 좀 들어라!!!

- 로맨틱 코미디로 방향을 잡았으면 평타는 쳤을텐데.. 개연성도 없는 이야기를 이끌고 가는게 좋은 배우로 이렇게 만들기 쉽지않은데

- 똥 싸다가 막힌 느낌 안보는거를 추천 드릴게요 안보시는걸 추천

- 어우 핵노잼 영화가 왜 이렇게 하급인건지

- 솔직히 재미없음 감독이 뭘 말할려고하는 의도인지 기괴스럽긴한데 사실 내용이 없음 그냥 망작임

- 뭘 말하고싶은지 모르겠다 아쉽다

- 새벽에 케이블에서 볼거 없어서 노잼이네요 참고 보다가 좃불씬에서 채널 돌림

- 핵노잼 억지 착한척 오졌다

- 오그라드는 대사는 기본에 옵션으로 발연기 장착! 비유하자면 못만든 밥에 그나마 금가루 뿌려놓은 듯. 최하 등급.

- 역대급 호화캐스팅인데 스토리, 개연성, 연출, 액션을 왜 이렇게 만들었는지 진짜 너무 아쉽다. 연기파 배우들만 오질나게 고생했다.

- 스토리 개 말도 안되는 억지 고구마 9억만 개 우리나라 경찰의 무능함을 잘 집어준건 인정

- 어설픈 연출력 어설픈 결말 현실감이라고는 단 1도 없는 영화는 진짜 오랜만이네... 돈 아깝. 시간 아깝

- 노잼 하 2시간 반 돌려내 이제 마블 영화는 걸러야겠다 나이먹으니 유치해서 못보겠다 중간에 너무 지루해서 잠 올 뻔

- 재미없음 충분히 더 흥미롭게 다룰 수 있는 주제였는데 이정도 밖에 못만든게 아쉽다 기대가 커서 더 아쉽다

- 생각보다 더 감흥이 없어요. 액션씬이 많고 소재가 신박하다고는 할 수 있지만 뻔한 느낌.. 배우 연기도 개인적으로 많이 아쉽다. 어떤 후기에서 킬링타임용으로 보기 좋다는 말을 봤는데 딱 그 정도네요ㅠ

- 그냥 보지마세요 쓰레기 영화임 노잼 발암 캐릭터 오지게 넣어서 강제즙 짜는 스토리 이걸 본 저도 한심함

- 지루해서 이거 다 보는데 두 달 넘게 걸린듯ㅋㅋㅋ 엄태구랑 전여빈 땜에 봤음. 그냥 킬링타임으로는 쏘쏘

Overwriting final_negative.txt


In [20]:
%%writefile final_positive.txt
- 진짜 꼭 보세요 최고의 영화

- 엔드게임 이후 마블이 한 번 더 뽑아낸 희대의 명작

- 정말 오랜만에 감동 울었네... 펑펑

- 연출에서 등장인물 하나하나의 아픔을 어루만져 주는 섬세함에 감동했습니다

- 어렸을때는 그냥 액션과 화려함에 좋아했는데 크고나서 보니 정말 하나하나 감동이였고 너무 재밌었습니다

- 꼭 보세여 넘 재밌어

- 어릴적 영화관에서 처음으로 본 영화가 스파이더맨이어서 이번 영화는 더 남달랐습니다. 뭉클하기고 했고 전투도 너무 좋고 올해 최고의 영화입니다!

- 올해 본 영화 중 최고의 영화입니다.

- 극장에서 보는 거 적극 추천 최고!

- 인생영화이고 평생 못잊을거같다

- 그냥 진짜 재밌어 진짜 잘 만든 영화.

- 자연과 인간, 아버지와 아들 등 공감하고 몰입하며 감동 받기에 충분한 이야기를 잘 전달하고 있다.

- 진정한 아름다움은 완벽이 아니라 상처에 있다고. 그 상처 외면하지 않을 때, 인간은 깊고 참된 사랑을 구현할 수 있다고 일러주는 영화. 함부로 상처와 화해시키지도, 갑자기 모든 걸 치유하지도 않는, 거칠고 다정한 마스터피스. 적극 추천합니다! 

- 띵작입니다 ㄹㅇ!!!

- 감동 그 자체 스파이더맨 짱

- 최고입니다. 여운이 가시지가 않습니다.

- 단언컨대 한국 최고의 영화 복수혈전에 버금가는최고의 영화 꼭 봐야되는 한국 영화!!

- 그냥 대박임 그 시절 스파이더맨 노스텔지아

- 다음 날 새벽 출근임에도 불구하고 심야영화를 본 게 아깝지 않다. 역대급 대박임

- 최고중 최고 진짜 베스트

- 보는 내내 명치끝이 탁 막혀서 쿠키엔딩도 없는데 여운이 너무 긴데다 음악을 듣느라 엔딩크래딧이 올라가는 걸 다 본 영화도 처음이고. 권장영화가 되었으면 좋겠다.

Overwriting final_positive.txt


* (3-1) 데이터프레임을 생성하세요.



In [21]:
p_rdd = spark.sparkContext.textFile('final_positive.txt')
p_rdd = p_rdd\
    .filter(lambda x: x != '')\
    .map(lambda x: x[2:])\
    .map(lambda x: [x, 1.])

df3_p = spark.createDataFrame(p_rdd, ['text', 'emotion'])
df3_p.show()

+----------------------------------+-------+
|                              text|emotion|
+----------------------------------+-------+
|        진짜 꼭 보세요 최고의 영화|    1.0|
|  엔드게임 이후 마블이 한 번 더...|    1.0|
| 정말 오랜만에 감동 울었네... 펑펑|    1.0|
|연출에서 등장인물 하나하나의 아...|    1.0|
|어렸을때는 그냥 액션과 화려함에...|    1.0|
|               꼭 보세여 넘 재밌어|    1.0|
| 어릴적 영화관에서 처음으로 본 ...|    1.0|
|올해 본 영화 중 최고의 영화입니다.|    1.0|
|  극장에서 보는 거 적극 추천 최고!|    1.0|
|    인생영화이고 평생 못잊을거같다|    1.0|
|  그냥 진짜 재밌어 진짜 잘 만든...|    1.0|
|  자연과 인간, 아버지와 아들 등...|    1.0|
|진정한 아름다움은 완벽이 아니라...|    1.0|
|                띵작입니다 ㄹㅇ!!!|    1.0|
|        감동 그 자체 스파이더맨 짱|    1.0|
| 최고입니다. 여운이 가시지가 않...|    1.0|
| 단언컨대 한국 최고의 영화 복수...|    1.0|
| 그냥 대박임 그 시절 스파이더맨...|    1.0|
| 다음 날 새벽 출근임에도 불구하...|    1.0|
|           최고중 최고 진짜 베스트|    1.0|
+----------------------------------+-------+
only showing top 20 rows



In [22]:
n_rdd = spark.sparkContext.textFile('final_negative.txt')
n_rdd = n_rdd\
    .filter(lambda x: x != '')\
    .map(lambda x: x[2:])\
    .map(lambda x: [x, 0.])

df3_n = spark.createDataFrame(n_rdd, ['text', 'emotion'])
df3_n.show()

+----------------------------------+-------+
|                              text|emotion|
+----------------------------------+-------+
|고구마 먹은 영화 유치해서 못보겠다|    0.0|
|  노잼이네요.. 왜 배우들이 출연...|    0.0|
|   제발 뻘짓 좀 그만하셨으면......|    0.0|
|         스토리가 매우 매우 아쉽다|    0.0|
|             영화 약간 이해가 안됨|    0.0|
|좋아하는 배우들인데 최근 영화라...|    0.0|
| 그지같네 도대체 이 따위 영화를...|    0.0|
|  스크린보다 시계를 더 많이 봄....|    0.0|
| 이때까지 본 영화중에 제일 최악...|    0.0|
|    나만 짜증.. 줄거리 한줄평.....|    0.0|
|로맨틱 코미디로 방향을 잡았으면...|    0.0|
| 똥 싸다가 막힌 느낌 안보는거를...|    0.0|
|  어우 핵노잼 영화가 왜 이렇게 ...|    0.0|
| 솔직히 재미없음 감독이 뭘 말할...|    0.0|
|   뭘 말하고싶은지 모르겠다 아쉽다|    0.0|
| 새벽에 케이블에서 볼거 없어서 ...|    0.0|
|         핵노잼 억지 착한척 오졌다|    0.0|
|오그라드는 대사는 기본에 옵션으...|    0.0|
| 역대급 호화캐스팅인데 스토리, ...|    0.0|
|  스토리 개 말도 안되는 억지 고...|    0.0|
+----------------------------------+-------+
only showing top 20 rows



In [23]:
df3 = df3_p.union(df3_n)
df3.show(80)

+----------------------------------+-------+
|                              text|emotion|
+----------------------------------+-------+
|        진짜 꼭 보세요 최고의 영화|    1.0|
|  엔드게임 이후 마블이 한 번 더...|    1.0|
| 정말 오랜만에 감동 울었네... 펑펑|    1.0|
|연출에서 등장인물 하나하나의 아...|    1.0|
|어렸을때는 그냥 액션과 화려함에...|    1.0|
|               꼭 보세여 넘 재밌어|    1.0|
| 어릴적 영화관에서 처음으로 본 ...|    1.0|
|올해 본 영화 중 최고의 영화입니다.|    1.0|
|  극장에서 보는 거 적극 추천 최고!|    1.0|
|    인생영화이고 평생 못잊을거같다|    1.0|
|  그냥 진짜 재밌어 진짜 잘 만든...|    1.0|
|  자연과 인간, 아버지와 아들 등...|    1.0|
|진정한 아름다움은 완벽이 아니라...|    1.0|
|                띵작입니다 ㄹㅇ!!!|    1.0|
|        감동 그 자체 스파이더맨 짱|    1.0|
| 최고입니다. 여운이 가시지가 않...|    1.0|
| 단언컨대 한국 최고의 영화 복수...|    1.0|
| 그냥 대박임 그 시절 스파이더맨...|    1.0|
| 다음 날 새벽 출근임에도 불구하...|    1.0|
|           최고중 최고 진짜 베스트|    1.0|
|  보는 내내 명치끝이 탁 막혀서 ...|    1.0|
|고구마 먹은 영화 유치해서 못보겠다|    0.0|
|  노잼이네요.. 왜 배우들이 출연...|    0.0|
|   제발 뻘짓 좀 그만하셨으면......|    0.0|
|         스토리가 매우 매우 아쉽다|    0.0|
|             영화 약간 이해가 안됨|    0.0|
|좋아하는 배우들

* (3-2) 불용어 제거, 불필요 단어 정리



In [24]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol='text', outputCol='words')
tokDf3 = tokenizer.transform(df3)
tokDf3.show()

+----------------------------------+-------+----------------------------------+
|                              text|emotion|                             words|
+----------------------------------+-------+----------------------------------+
|        진짜 꼭 보세요 최고의 영화|    1.0|     [진짜, 꼭, 보세요, 최고의,...|
|  엔드게임 이후 마블이 한 번 더...|    1.0|    [엔드게임, 이후, 마블이, 한...|
| 정말 오랜만에 감동 울었네... 펑펑|    1.0|    [정말, 오랜만에, 감동, 울었...|
|연출에서 등장인물 하나하나의 아...|    1.0|  [연출에서, 등장인물, 하나하나...|
|어렸을때는 그냥 액션과 화려함에...|    1.0|    [어렸을때는, 그냥, 액션과, ...|
|               꼭 보세여 넘 재밌어|    1.0|          [꼭, 보세여, 넘, 재밌어]|
| 어릴적 영화관에서 처음으로 본 ...|    1.0|  [어릴적, 영화관에서, 처음으로...|
|올해 본 영화 중 최고의 영화입니다.|    1.0|      [올해, 본, 영화, 중, 최고...|
|  극장에서 보는 거 적극 추천 최고!|    1.0|     [극장에서, 보는, 거, 적극,...|
|    인생영화이고 평생 못잊을거같다|    1.0|[인생영화이고, 평생, 못잊을거같다]|
|  그냥 진짜 재밌어 진짜 잘 만든...|    1.0|     [그냥, 진짜, 재밌어, 진짜,...|
|  자연과 인간, 아버지와 아들 등...|    1.0|     [자연과, 인간,, 아버지와, ...|
|진정한 아름다움은 완벽이 아니라...|    1.0|   [진정한, 아름다움은, 완벽이,...|
|              

In [25]:
stopwords = [
    u"또",
    u"위해",
    u"참",
    u"더",
    u"그러나",
    u"그",
    u"특히",
    u"느낌표", u"!",
    u"점", u".",
    u"ㅋ", u"ㅋㅋ", u"ㅋㅋㅋ", u"ㅋㅋㅋㅋ", u"ㅋㅋㅋㅋㅋ", u"ㅋㅋㅋㅋㅋㅋㅋ",
    u"ㅠ", u"ㅠㅠ", u"ㅠㅠㅠ", u"ㅠㅠㅠㅠ", u"ㅠㅠㅠㅠㅠ", u"ㅠㅠㅠㅠㅠㅠ",
    u"ㄹㅇ", u"그", u"수", u"본", u"등", u"때", u"걸", u"다", u"게",
    u"날", u"짱", u"거", u"중", u"더", u"넘", u"두", u"달", u"을", u"뻔",
    u"단", u"개", u"듯",
]

from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol='words', outputCol='nostopwords')
stop.setStopWords(stopwords)

stopDf3 = stop.transform(tokDf3)
stopDf3.show()

21/12/18 22:36:04 WARN StopWordsRemover: Default locale set was [en_KR]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


+----------------------------------+-------+----------------------------------+----------------------------------+
|                              text|emotion|                             words|                       nostopwords|
+----------------------------------+-------+----------------------------------+----------------------------------+
|        진짜 꼭 보세요 최고의 영화|    1.0|     [진짜, 꼭, 보세요, 최고의,...|     [진짜, 꼭, 보세요, 최고의,...|
|  엔드게임 이후 마블이 한 번 더...|    1.0|    [엔드게임, 이후, 마블이, 한...|    [엔드게임, 이후, 마블이, 한...|
| 정말 오랜만에 감동 울었네... 펑펑|    1.0|    [정말, 오랜만에, 감동, 울었...|    [정말, 오랜만에, 감동, 울었...|
|연출에서 등장인물 하나하나의 아...|    1.0|  [연출에서, 등장인물, 하나하나...|  [연출에서, 등장인물, 하나하나...|
|어렸을때는 그냥 액션과 화려함에...|    1.0|    [어렸을때는, 그냥, 액션과, ...|    [어렸을때는, 그냥, 액션과, ...|
|               꼭 보세여 넘 재밌어|    1.0|          [꼭, 보세여, 넘, 재밌어]|              [꼭, 보세여, 재밌어]|
| 어릴적 영화관에서 처음으로 본 ...|    1.0|  [어릴적, 영화관에서, 처음으로...|  [어릴적, 영화관에서, 처음으로...|
|올해 본 영화 중 최고의 영화입니다.|    1.0|      [올해, 본, 영화, 중, 최고...|    [올해, 영화, 최고의, 영

* (3-3) TF-IDF를 계산

In [26]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol='nostopwords', outputCol='hash')
hashDf3 = hashTF.transform(stopDf3)

idf = IDF(inputCol='hash', outputCol='tf-idf')
idfModel = idf.fit(hashDf3)
tfidfDf3 = idfModel.transform(hashDf3)

tfidfDf3.select(['nostopwords', 'hash', 'tf-idf']).show()

21/12/18 22:36:11 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


+----------------------------------+--------------------+--------------------+
|                       nostopwords|                hash|              tf-idf|
+----------------------------------+--------------------+--------------------+
|     [진짜, 꼭, 보세요, 최고의,...|(262144,[3788,173...|(262144,[3788,173...|
|    [엔드게임, 이후, 마블이, 한...|(262144,[11835,90...|(262144,[11835,90...|
|    [정말, 오랜만에, 감동, 울었...|(262144,[63945,66...|(262144,[63945,66...|
|  [연출에서, 등장인물, 하나하나...|(262144,[48058,50...|(262144,[48058,50...|
|    [어렸을때는, 그냥, 액션과, ...|(262144,[14924,59...|(262144,[14924,59...|
|              [꼭, 보세여, 재밌어]|(262144,[53566,12...|(262144,[53566,12...|
|  [어릴적, 영화관에서, 처음으로...|(262144,[2549,173...|(262144,[2549,173...|
|    [올해, 영화, 최고의, 영화입...|(262144,[17393,14...|(262144,[17393,14...|
|    [극장에서, 보는, 적극, 추천...|(262144,[13247,31...|(262144,[13247,31...|
|[인생영화이고, 평생, 못잊을거같다]|(262144,[70887,13...|(262144,[70887,13...|
|     [그냥, 진짜, 재밌어, 진짜,...|(262144,[14809,85...|(262144,[14809,85...|
|     [

* (3-4) 베이지안 모델을 적용해서 분류 (alpha는 1.0으로 설정)

In [27]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(
    featuresCol='tf-idf', 
    labelCol='emotion', 
    smoothing=1.0, # alpha 
    modelType='multinomial', 
    predictionCol='prediction')

model = nb.fit(tfidfDf3)

21/12/18 22:36:16 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/12/18 22:36:16 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


In [28]:
nbDf3 = model.transform(tfidfDf3)
nbDf3.select(['text', 'emotion', 'prediction']).show(40)

21/12/18 22:36:21 WARN DAGScheduler: Broadcasting large task binary with size 8.1 MiB
21/12/18 22:36:21 WARN DAGScheduler: Broadcasting large task binary with size 8.1 MiB


+----------------------------------+-------+----------+
|                              text|emotion|prediction|
+----------------------------------+-------+----------+
|        진짜 꼭 보세요 최고의 영화|    1.0|       1.0|
|  엔드게임 이후 마블이 한 번 더...|    1.0|       1.0|
| 정말 오랜만에 감동 울었네... 펑펑|    1.0|       1.0|
|연출에서 등장인물 하나하나의 아...|    1.0|       1.0|
|어렸을때는 그냥 액션과 화려함에...|    1.0|       1.0|
|               꼭 보세여 넘 재밌어|    1.0|       1.0|
| 어릴적 영화관에서 처음으로 본 ...|    1.0|       1.0|
|올해 본 영화 중 최고의 영화입니다.|    1.0|       1.0|
|  극장에서 보는 거 적극 추천 최고!|    1.0|       1.0|
|    인생영화이고 평생 못잊을거같다|    1.0|       1.0|
|  그냥 진짜 재밌어 진짜 잘 만든...|    1.0|       1.0|
|  자연과 인간, 아버지와 아들 등...|    1.0|       1.0|
|진정한 아름다움은 완벽이 아니라...|    1.0|       1.0|
|                띵작입니다 ㄹㅇ!!!|    1.0|       1.0|
|        감동 그 자체 스파이더맨 짱|    1.0|       1.0|
| 최고입니다. 여운이 가시지가 않...|    1.0|       1.0|
| 단언컨대 한국 최고의 영화 복수...|    1.0|       1.0|
| 그냥 대박임 그 시절 스파이더맨...|    1.0|       1.0|
| 다음 날 새벽 출근임에도 불구하...|    1.0|       1.0|
| 

* (3-5) 훈련, 테스트 데이터를 반반씩 분리하고, 훈련 후 정확성을 계산하고 출력.

In [29]:
(tfidfDf3_train, tfidfDf3_test) = tfidfDf3.randomSplit([0.5, 0.5])

In [30]:
model = nb.fit(tfidfDf3_train)

nbDf3_test = model.transform(tfidfDf3_test)
nbDf3_test.select(['text', 'emotion', 'prediction']).show(40)

21/12/18 22:36:25 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/12/18 22:36:26 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/12/18 22:36:28 WARN DAGScheduler: Broadcasting large task binary with size 8.1 MiB
21/12/18 22:36:28 WARN DAGScheduler: Broadcasting large task binary with size 8.1 MiB


+----------------------------------+-------+----------+
|                              text|emotion|prediction|
+----------------------------------+-------+----------+
| 그냥 대박임 그 시절 스파이더맨...|    1.0|       1.0|
|  그냥 진짜 재밌어 진짜 잘 만든...|    1.0|       0.0|
|  극장에서 보는 거 적극 추천 최고!|    1.0|       0.0|
|               꼭 보세여 넘 재밌어|    1.0|       1.0|
| 다음 날 새벽 출근임에도 불구하...|    1.0|       0.0|
| 단언컨대 한국 최고의 영화 복수...|    1.0|       1.0|
|                띵작입니다 ㄹㅇ!!!|    1.0|       0.0|
|어렸을때는 그냥 액션과 화려함에...|    1.0|       0.0|
| 어릴적 영화관에서 처음으로 본 ...|    1.0|       1.0|
|  자연과 인간, 아버지와 아들 등...|    1.0|       0.0|
|진정한 아름다움은 완벽이 아니라...|    1.0|       0.0|
|           최고중 최고 진짜 베스트|    1.0|       0.0|
|고구마 먹은 영화 유치해서 못보겠다|    0.0|       0.0|
| 그냥 보지마세요 쓰레기 영화임 ...|    0.0|       0.0|
| 그지같네 도대체 이 따위 영화를...|    0.0|       1.0|
| 솔직히 재미없음 감독이 뭘 말할...|    0.0|       0.0|
|         스토리가 매우 매우 아쉽다|    0.0|       0.0|
| 어설픈 연출력 어설픈 결말 현실...|    0.0|       0.0|
|  어우 핵노잼 영화가 왜 이렇게 ...|    0.0|       0.0|


In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="emotion", labelCol="prediction")
acc = evaluator.evaluate(nbDf3_test)

print(f"정확성: {acc:.5f}")

21/12/18 22:36:30 WARN DAGScheduler: Broadcasting large task binary with size 8.1 MiB


정확성: 0.63333


In [32]:
%%writefile final_test.txt
- 고구가 먹은 느낌 킬링타음으로도 아쉽다

- 내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다

- 보다가 채널 돌림 재미없다

- 최고의 영화

- 여운이 긴 명작

- 띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요

Overwriting final_test.txt


In [33]:
_rdd = spark.sparkContext.textFile('final_test.txt')
_rdd = _rdd\
    .filter(lambda x: x != '')\
    .map(lambda x: [x[2:]])

testDf3 = spark.createDataFrame(_rdd, ['text'])

tokenizer = Tokenizer(inputCol='text', outputCol='words')
testDf3 = tokenizer.transform(testDf3)

testDf3 = stop.transform(testDf3)

hashTF = HashingTF(inputCol='nostopwords', outputCol='hash')
testDf3 = hashTF.transform(testDf3)

idf = IDF(inputCol='hash', outputCol='tf-idf')
idfModel = idf.fit(testDf3)
testDf3 = idfModel.transform(testDf3)

testDf3 = model.transform(testDf3)
testDf3.select(['text', 'prediction']).show(10)

+----------------------------------+----------+
|                              text|prediction|
+----------------------------------+----------+
|고구가 먹은 느낌 킬링타음으로도...|       0.0|
| 내돈 아깝 핵노잼 영화가 이렇게...|       0.0|
|         보다가 채널 돌림 재미없다|       0.0|
|                       최고의 영화|       1.0|
|                    여운이 긴 명작|       1.0|
|띵작입니다 몰입하고 숨도 못쉬고...|       1.0|
+----------------------------------+----------+



21/12/18 22:36:37 WARN DAGScheduler: Broadcasting large task binary with size 8.0 MiB


* (3-6) 결론

- 고구가 먹은 느낌 킬링타음으로도 아쉽다 -> 부정

- 내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다 -> 부정

- 보다가 채널 돌림 재미없다 -> 부정

- 최고의 영화 -> 긍정

- 여운이 긴 명작 -> 긍정

- 띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요 -> 긍정

모두 맞추었다!